In [1]:
import requests
import json
import numpy as np
import os
import glob
import time
from datetime import datetime

In [2]:
from tqdm import tqdm

In [3]:
parts_url 
feed_url_mask
parts_per_page = 100

In [4]:
max_attempt_count = 5 # столько раз на каждый запрос
max_data_is_empty_attempt_count = 3 #если столько раз подряд пусто, значит кончилась лента
max_page=99
results_folder = 'results'
parts_results_folder = 'parts_results'
if not os.path.exists(results_folder):
    os.makedirs(results_folder)
if not os.path.exists(os.path.join(results_folder, parts_results_folder)):
    os.makedirs(os.path.join(results_folder, parts_results_folder))

In [5]:
#clean folder
if datetime.utcnow().hour==10:
    if os.path.isfile(os.path.join(results_folder, "parts.json")):
        os.remove(os.path.join(results_folder, "parts.json"))

    for f in os.listdir(os.path.join(results_folder, parts_results_folder)):
        if f.endswith(".json"):
            os.remove(os.path.join(results_folder, parts_results_folder, f))

In [6]:
#parse part types
if not os.path.exists(os.path.join(results_folder, "parts.json")):
    attempt = 1
    status_code = 0

    while status_code!=200 and attempt < max_attempt_count:
      r = requests.get(url=parts_url)
      status_code = r.status_code
      if r.status_code==200:
        part_types = json.loads(r.text[r.text.index('(')+1 : -2])
        with open(os.path.join(results_folder, "parts.json"), 'w', encoding='utf8') as outfile:
            json.dump(part_types, outfile, ensure_ascii=False, indent=4)
      else:
        time.sleep(5)
        attempt+=1

    if attempt == max_attempt_count:
        mess = 'error while parse part types, last_status:{0}'.format(status_code)
        raise Exception(mess)
else:
    with open(os.path.join(results_folder, "parts.json"), 'r', encoding="utf8") as json_file:
        part_types = json.load(json_file)

In [7]:
#read companies if exists
if os.path.exists(os.path.join(results_folder, parts_results_folder, 'companies.json')):
    with open(os.path.join(results_folder, parts_results_folder, 'companies.json'), 'r', encoding="utf8") as json_file:
        company_dict = json.load(json_file)
else:
    company_dict={}

In [ ]:
#parse feed
try:
    for part_type in tqdm(part_types):
        part_type = part_type[0]
        if part_type in [45,49,61]:
            continue
        if not os.path.exists(os.path.join(results_folder, parts_results_folder, str(part_type) + '.json')):
            page = 1
            price_from = 0
            is_end_of_feed = False
            data_is_empty_attempt = 1
            result_list=[]
            #проверим последнюю страницу. Если данные есть, заберем последнюю цену
            
            
            #все страницы
            while not is_end_of_feed:
                attempt = 1
                status_code = 0
                data_is_empty = False
                while status_code!=200 and attempt < max_attempt_count:
                    url = feed_url_mask.format(part_type, page, parts_per_page, price_from)
                    if page==1:
                        url = url.replace('&page=1','')
                    if price_from==0:
                        url = url.replace('&price_from=0','')
                    #print(url)
                    try:
                        r = requests.get(url=url)
                        status_code = r.status_code
                    except:
                        status_code = -1
                    if r.status_code==200:
                        json_response = json.loads(r.text)
                        if(len(json_response['data']) == 0):
                            data_is_empty_attempt+=1
                            data_is_empty=True
                            break
                        data = json_response['data']
                        for item in data:
                            item.update( {"ts":int(time.time()), "page":page, "price_from":price_from})
                        result_list.extend(data)
                        company_dict.update(json_response['company'])
                    else:
                        time.sleep(5)
                        attempt+=1
                if attempt == max_attempt_count:
                    mess = 'error while parse parts, part_type:{0},  last_status:{0}'.format(part_type, status_code)
                    raise Exception(mess)
                if data_is_empty:
                    if data_is_empty_attempt==3:
                        is_end_of_feed = True
                    continue
                
                if page==max_page:
                    page = 1
                    price_from = max([x['price'] for x in data])
                else:
                    page+=1

            with open(os.path.join(results_folder, parts_results_folder, str(part_type) + '.json'), 'w', encoding='utf8') as outfile:
                json.dump(result_list, outfile, ensure_ascii=False, indent=4)
            with open(os.path.join(results_folder, parts_results_folder, 'companies.json'), 'w', encoding='utf8') as outfile:
                json.dump(company_dict, outfile, ensure_ascii=False, indent=4)
        
except Exception as e:
    with open(os.path.join(results_folder, parts_results_folder, 'companies.json'), 'w', encoding='utf8') as outfile:
        json.dump(company_dict, outfile, ensure_ascii=False, indent=4)
    raise e

  2%|█▎                                                                             | 14/867 [07:27<4:53:53, 20.67s/it]

In [ ]:
len(result_list)

In [ ]:
part_type

In [ ]:
attempt

In [ ]:
 os.listdir(os.path.join(results_folder, parts_results_folder))

In [ ]:
files

In [ ]:
from collections import Counter
from itertools import groupby

f = lambda x: x['sup_invnn']
dct = {k: Counter(d['sup_invnn'] for d in g) for k, g in groupby(result_list, f)}
print(dct)

In [ ]:
len(result_list)

In [ ]:
from collections import Counter

c = Counter([d['sup_invnn'] for d in result_list])
print(c)

In [ ]:
[d['page'] for d in result_list if d['sup_invnn'] == '39579-39646-93S33J301']

In [ ]:
from collections import Counter

c = Counter([d['spt_translite'] for d in result_list])
print(c)

In [ ]:
#count
part_type = 31801
page = 1

attempt = 1
status_code = 0

while status_code!=200 and attempt < max_attempt_count:
    r = requests.get(url=feed_url_mask.format(part_type, page, parts_per_page))
    status_code = r.status_code
    if r.status_code==200:
        json_response = json.loads(r.text)
        print(json_response["count"])
    else:
        time.sleep(5)
        attempt+=1

if attempt == max_attempt_count:
  print('last_status:{0}'.format(status_code))